##### *Python libraries used in this notebook*

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as wg
from IPython.display import display
from scipy.integrate import odeint

# PID control

## Interactive simulation of PID controller tuning

### Abstract
This notebook discusses what control systems are and what types exist. What are the different mathematical methods and devices for controlling these systems. What are PID controllers and how to tune them. Simulations are shown to make it clearer to readers about the setup of PID controllers and the management of processes and objects.

### Table of Contents

* [1. Description of Control System](#chapter1)
    * [1.1. Description of Control Loop](#section_1_1)
        * [1.1.1. Open Loop Control System](#section_1_1_1)
        * [1.1.2. Closed Loop Control System](#section_1_1_2)
* [2. PID Controller introduction](#chapter2)
    * [2.1. PID Controller theory](#section_2_1)
        * [2.1.1. Interactive algorithm](#section_2_1_1)
        * [2.1.2. Noninteractive algorithm](#section_2_1_2)
        * [2.1.3. Parallel algorithm](#section_2_1_3)
    * [2.2. PID Controller terms](#section_2_2)
        * [2.2.1. Proportional term ](#section_2_2_1)
        * [2.2.2. Integral component](#section_2_2_2)
        * [2.2.3. Derivative component](#section_2_2_3)
    * [2.3. PID Controller tuning methods](#section_2_3)
        * [2.3.1. Ziegler-Nichols Method](#section_2_3_1)
        * [2.3.2. Cohen-Coon Method](#section_2_3_2)
        * [2.3.3. Tyreus-Luyben Method](#section_2_3_3)
        * [2.3.4. IAE, ITAE Method](#section_2_3_4)
    * [2.4. Dynamic process response - FOPDT model](#section_2_4)
        * [2.4.1. Process gain](#section_2_4_1)
        * [2.4.2. Process time constant](#section_2_4_2)
        * [2.4.3. Process time delay](#section_2_4_3)
        * [2.4.4. Interactive simulation - FOPDT model](#section_2_4_4)
* [3. Interactive simulations of PID controller tuning](#chapter3)
    * [3.1. P control tuning](#section_3_1)
    * [3.2. PI control tuning](#section_3_2)
    * [3.3. PID control tuning](#section_3_3)
* [4. Conclusion](#chapter4)
* [5. References](#chapter5)

### 1. Description of Control System <a class="anchor" id="chapter1"></a>
A **Control System** is a system that provides the desired response by controlling the output. Explained in another way, this is a set of mechanical and electronic devices which regulate, manage, and command other devices or systems using control loops. The following figure shows the simple block diagram of a control system.

<img src="./imgs/ControlSystemBlockDiagram.jpg" width="500" />

Controls are classified with respect to:
> - technique involved to perform control (i.e. human/machines): manual/automatic control
> - creation of control system (i.e. natural / manmade): natural/ manmade /combinational
control
> - Time dependence of output variable (i.e. constant/changing): regulator/servo, (also known as
regulating/tracking control)
> - Time-type control (Continuous/ discrete): Continuous/Discrete time control
> - Linearity of the control system (linear/nonlinear): Linear/nonlinear control system
> - Based on the number of input output (SISO, MISO) system
> - fundamental structure / configuration of the control (i.e. the information used for computing the
control): open loop/feedback control, (also known as open loop/closed loop control)
    
This definition makes sense if you know what a **Control Loop** is, so let's define what is a control loop.

### 1.1. Description of Control Loop <a class="anchor" id="section_1_1"></a>
A **Control Loop** is the configuration of the control system. This configuration consists of all the physical components and control functions needed to adjust or maintain the value of a measured process variable $PV$ to your desired value, or setpoint $SP$. To do this, your loop needs at least three basic elements – an input, a controller, and an output. The following figures show some process control loops in modern industrial automation.

<img src="./imgs/ControlLoopExample.jpg" width="1000" />

There are two types of control loops: 
> - Open Loop
> - Closed Loop

Let's look at these two types.

### 1.1.1. Open Loop Control System <a class="anchor" id="section_1_1_1"></a>
An **Open Loop Control System** is applied to achieve the desired system response using a controller or an actuator without feedback. This means that the оpen loop systems do not monitor or correct the output for disturbances. The desired process response is applied to a controller and it produces an actuating signal or controlling signal (output). This signal is given as an input to a plant or process which is to be controlled. So, the plant produces an output, which is controlled. The following figure shows the simple block diagram of a control system.

<img src="./imgs/OpenLoop.jpg" width="600" />

Advantages of open Loop control:
> - These systems are simple in construction and design.
> - These systems are economical.
> - These systems are easy from a maintenance point of view.
> - Usually, these systems are not much troubled with problems of stability.
> - These systems are convenient to use when output is difficult to measure.

Disadvantages of open loop control:
> - These systems are not accurate and reliable because of their accuracy depends on the accuracy of
the calibration.
> - In these systems, inaccuracy results are obtained with parameter variations i.e. internal
disturbances.
> - Recalibration of the controller is required from time to time for maintaining quality and accuracy.

### 1.1.2. Closed Loop Control System <a class="anchor" id="section_1_1_2"></a>
A **Closed Loop Control System** is used to achieve the desired system response using a controller with the output measurement as a feedback signal. The use of feedback enables us to improve system performance at the cost of introducing the measurement noise and stability problem. This means that the closed loop systems monitor the output and compare it to the input. If an error is detected, the system corrects the output and hence corrects the effects of disturbances. The following figure shows the simple block diagram of a control system.

<img src="./imgs/ClosedLoop.jpg" width="500" />

Advantages of closed loop control:
> - In these systems, accuracy is very high due to the correction of any arising error.
> - Since these systems sense environmental changes as well as internal disturbances, the errors are
modified.
> - There is a reduced effect of nonlinearity in these systems.
> - Systems have high bandwidth i.e. high operating frequency zone.
> - There are facilities for automation in these systems.

Disadvantages of closed loop control:
> - Systems are complicated in design and, hence, costlier.
> - Systems may be unstable.

After we already know what a control system is and what types of configurations it has, maybe it is time to understand what mechanisms we use in modern process industrial automation. Let's look at what a PID controller is.

### 2. PID Controller introduction <a class="anchor" id="chapter2"></a>
A **Proportional-Integral-Derivative Controller (PID Controller)** is a control loop feedback mechanism widely used in industrial control systems. A PID controller calculates an error value as the difference between a measured process variable and a desired setpoint. The controller attempts to minimize the error by adjusting the process through use of a manipulated variable. The following photos show PID controllers used in the industry

<img src="./imgs/PIDControllers.jpg" width="500" />

The PID controller algorithm involves three separate constant parameters, and is accordingly sometimes called **three-term control**: the proportional, the integral and derivative values, denoted $P$, $I$, and $D$. Simply put, these values can be interpreted in terms of time: $P$ depends on the ***present*** error, $I$ on the accumulation of ***past*** errors, and $D$ is a prediction of ***future*** errors, based on current rate of change. The weighted sum of these three actions is used to adjust the process via a control element such as the position of a control valve, a damper, or the power supplied to a heating element.

In the absence of knowledge of the underlying process, a PID controller has historically been considered to be the most useful controller. By tuning the three parameters in the PID controller algorithm, the controller can provide control action designed for specific process requirements. The response of the controller can be described in terms of the responsiveness of the controller to an error, the degree to which the controller overshoots the setpoint, and the degree of system oscillation. Note that the use of the PID algorithm for control does not guarantee optimal control of the system or system stability.

Some applications may require using only one or two actions to provide the appropriate system control. This is achieved by setting the other parameters to zero. A PID controller will be called a $P$, $PI$, $PD$ or $PID$ controller in the absence of the respective control actions. $PI$ controllers are fairly common, since derivative action is sensitive to measurement noise, whereas the absence of an integral term may prevent the system from reaching its target value due to the control action.

Let's see  the fundamentals behind the design of every PID controller.

### 2.1. PID Controller theory <a class="anchor" id="section_2_1"></a>
PID **Control Theory** is a branch of mathematics and engineering with the purpose of modifying the behavior of a dynamic system with input (e.g. input from a temperature sensor) that changes over time. The main objective of control theory is stability. Usually, (in closed loop systems) stability is achieved by continually taking measurements and making adjustments to the system.

The PID control scheme is named after its three correcting terms, whose sum constitutes the manipulated variable $MV$. The proportional, integral, and derivative terms are summed to calculate the output of the PID controller. Controller manufacturers arrange the Proportional, Integral, and Derivative modes into three different controller algorithms or controller structures. These are called Interactive, Noninteractive, and Parallel algorithms. Some controller manufacturers allow you to choose between different controller algorithms as a configuration option in the controller software. The PID Algorithms are:

### 2.1.1. Interactive algorithm <a class="anchor" id="section_2_1_1"></a>
The oldest controller algorithm is called the Series, Classical, Real or Interactive algorithm.  The original pneumatic and electronic controllers had this algorithm and it is still found it in many controllers today.  The Ziegler-Nichols PID tuning rules were developed for this controller algorithm.

<img src="./imgs/series.png" width="600" />

$$\large{u(t) = K_c \left[ e(t) + \frac{1}{\tau_I}\int_0^t e(\tau)\ d\tau\right] + \left[1 + \tau_D\frac{d}{dt} e(t)\right]\tag{1}}$$

**Where:**

$u(t)$ - Controller output

$\tau_I$ - Integral time constant

$\tau_D$ - Derivative time constant

$K_c$ - Controller gain

$e(t) = SP - PV$ - Control error, which is the difference between the desired process value, and the measured process value

### 2.1.2. Noninteractive algorithm <a class="anchor" id="section_2_1_2"></a>
The Noninteractive algorithm is also called the Ideal, Standard or ISA algorithm. The Cohen-Coon and Lambda PID tuning rules were designed for this algorithm. If no derivative is used (i.e. Td = 0), the interactive and noninteractive controller algorithms are identical.

<img src="./imgs/ideal.png" width="480" />

$$\large{u(t) = K_c \left[ e(t) + \frac{1}{\tau_I}\int_0^t e(\tau)\ d\tau + \tau_D\frac{d}{dt} e(t)\right]\tag{2}}$$

### 2.1.3. Parallel algorithm <a class="anchor" id="section_2_1_3"></a>
Some academic textbooks discuss the parallel form of PID controller, but it is also used in some DCSs and PLCs. This algorithm is simple to understand, but not intuitive to tune. The reason is that it has no controller gain (affecting all three control modes), it has a proportional gain instead (affecting only the proportional mode). Adjusting the proportional gain should be supplemented by adjusting the integral and derivative settings at the same time. Try to not use this controller algorithm if possible (in some DCSs it is an option, so select the alternative).

<img src="./imgs/parallel.png" width="450" />

$$\large{u(t) = K_p e(t) + K_i\int_0^t e(\tau)\ d\tau + K_d\frac{d}{dt} e(t)\tag{3}}$$

**Where:**

$K_p = K_c$ - Propotional gain

$K_i = \frac{K_c}{\tau_I}$ - Integral gain

$K_d = K_c \tau_D$ - Derivative gain

### 2.2. PID Controller terms <a class="anchor" id="section_2_2"></a>
In the following cells, we will look at the three components (terms) of each PID controller.

### 2.2.1. Proportional term <a class="anchor" id="section_2_2_1"></a>
The **Proportional term (component)** produces an output value that is proportional to the current error value. The proportional response can be adjusted by multiplying the error by a constant $K_p$, called the proportional gain constant. The proportional component is given by:

$$\large{u(t) = K_p e(t)}\tag{4}$$

If the Laplace transform is applied on both sides:

$$\large{U(s) = K_p E(s)}\tag{5}$$

$$\large{\frac{U(s)}{E(s)} = K_p}\tag{5.1}$$

Therefore, the transfer function of the proportional term is:

$$\large{K_p}\tag{6}$$

**Where:**

$U(s)$ - Laplace transform of the actuating signal $u(t)$

$E(s)$ - Laplace transform of the error signal $e(t)$

$K_p$ - Proportional gain

A high proportional gain results in a large change in the output for a given change in the error. If the proportional gain is too high, the system can become unstable. In contrast, a small gain results in a small output response to a large input error, and a less responsive or less sensitive controller. If the proportional gain is too low, the control action may be too small when responding to system disturbances. Tuning theory and industrial practice indicate that the proportional term should contribute to the bulk of the output change.

### 2.2.2. Integral component <a class="anchor" id="section_2_2_2"></a>
The contribution from the **Integral term (component)** is proportional to both the magnitude of the error and the duration of the error. The integral in a PID controller is the sum of the instantaneous error over time and gives the accumulated offset that should have been corrected previously. The accumulated error is then multiplied by the integral gain $K_i$ and added to the controller output. The integral component is given by:

$$\large{u(t) = K_i\int_0^t e(\tau)\ d\tau\tag{7}}$$

If the Laplace transform is applied on both sides:

$$\large{U(s) = \frac{K_i E(s)}{s}\tag{8}}$$

$$\large{\frac{U(s)}{E(s)} = \frac{K_i}{s}\tag{8.1}}$$

Therefore, the transfer function of the integral term is:

$$\large{\frac{K_i}{s}\tag{9}}$$

**Where:**

$K_i$ - Integral gain

The integral component accelerates the movement of the process towards set-point and eliminates the residual steady-state error that occurs with a pure proportional controller. However, since the integral component responds to accumulated errors from the past, it can cause the present value to overshoot the set point value.

### 2.2.3. Derivative component <a class="anchor" id="section_2_2_3"></a>
The derivative of the process error is calculated by determining the slope of the error over time and multiplying this rate of change by the derivative gain $K_d$. The magnitude of the contribution of the derivative component to the overall control action is termed the derivative gain. The derivative component is given by:

$$\large{u(t) = K_d\frac{d}{dt} e(t)\tag{10}}$$

If the Laplace transform is applied on both sides:

$$\large{U(s) = K_d s E(s)\tag{11}}$$

$$\large{\frac{U(s)}{E(s)} = K_d s\tag{11.1}}$$

Therefore, the transfer function of the derivative term is:

$$\large{K_d s\tag{12}}$$

**Where:**

$K_d$ - Derivative gain

Derivative action predicts system behavior and thus improves the settling time and stability of the system. An ideal derivative is not causal, so that implementations of PID controllers include an additional low pass filtering for the derivative component, to limit the high frequency gain and noise. Derivative action is seldom used in practice though (by one estimate in only 20% of deployed controllers), because of its variable impact on system stability in real world applications.

### 2.3. PID Controller tuning methods <a class="anchor" id="section_2_3"></a>
How to tune a controller for any control application quickly and appropriately? This question raised in 1942 is still up to date and constantly occupies the automation community worldwide. The answer is very intricate. This intricacy is comparable with the open hitherto unresolved Hilbert problems known from mathematics.

There are dozens of methods. They are classified mainly into **Classical methods** and **Innovative methods**. The classical methods are based on mathematical models that are created before the system starts. Innovative methods or "self-touch methods" are based on pre-implemented software and sensors that automatically tuning the controller during system operation. 

The tuning method must be chosen depending on the type of the control system (process control, robotic manipulations, electric drives, energy applications, biomedical applications, chemical applications, mechanical systems, HVAC, etc.). Some tuning methods are suitable for a particular control system, others are not. Therefore, before choosing a tuning method, it is good to be clear about exactly what object or process will need to be controlled.

In the following cells we will look at some of the most used classicla methods and rules.

### 2.3.1. Ziegler-Nichols Method <a class="anchor" id="section_2_3_1"></a>
In 1942 Ziegler and Nichols, both employees of Taylor Instruments, described simple mathematical procedures for tuning PID controllers. This procedure is now accepted as a standard in control systems practice.

This is a heuristic method and  It is performed by setting the $K_i$ and $K_d$ gains to zero. Then $K_p$ gain increased from zero until it reaches the ultimate gain $K_{cu}$, at which the output of the control loop has stable and consistent oscillations. $K_{cu}$ and the oscillation ultimate period $P_u$ are then used to set the $P$, $I$, and $D$ terms depending on the type of controller used and behaviour desired.

The rules of this method are summarized in the following table:

| Control Mode | Controller gain $K_c$ | Integral time $\tau_I$ | Derivative time $\tau_D$ |
|:------------:|:---------------------:|:----------------------:|:------------------------:|
| $P$          | $0.5 K_{cu}$          |                        |                          |
| $PI$         | $0.45 K_{cu}$         | $\frac{P_u}{1.2}$      |                          |
| $PID$        | $0.6 K_{cu}$          | $\frac{P_u}{2}$        | $\frac{P_u}{8}$          |

### 2.3.2. Cohen-Coon Method <a class="anchor" id="section_2_3_2"></a>
The Cohen-Coon tuning method isn’t suitable for every application. It can be used only on self-regulating processes. Most control loops (flow, temperature, pressure, speed, and composition) are, at least to some extent, self-regulating processes. These processes always stabilize at some point of equilibrium, which depends on the process design and the controller output. If the controller output is set to a different value, the process will respond and stabilize at a new point of equilibrium.

The rules of this method are summarized in the following table:

| Control Mode | Controller gain $K_c$ | Integral time $\tau_I$ | Derivative time $\tau_D$ |
|:------------:|:---------------------:|:----------------------:|:------------------------:|
| $P$          | $P_g$                 |                        |                          |
| $PI$         | $PI_g$                | $\alpha$               |                          |
| $PID$        | $PID_g$               | $\beta$                | $\gamma$                 |

**Where:**

$\tau_p$ - Process time constant

$\theta_p$ - Process dead time

$P_g = \frac{\Big(1.03 + 0.35 \big(\frac{\tau_p}{\theta_p}\big)\Big)\tau_p}{\tau_p - \theta_p}$

$PI_g = \frac{\Big(1.9 + 0.083 \big(\frac{\tau_p}{\theta_p}\big)\Big)\tau_p}{\tau_p - \theta_p}$

$PID_g = \frac{\Big(1.35 + 0.25 \big(\frac{\tau_p}{\theta_p}\big)\Big)\tau_p}{\tau_p - \theta_p}$

$\alpha = \frac{\Big(0.9 + 0.083 \big(\frac{\tau_p}{\theta_p}\big)\Big)\theta_p}{1.27 + 0.6\big(\frac{\tau_p}{\theta_p}\big)}$

$\beta = \frac{\Big(1.35 + 0.25 \big(\frac{\tau_p}{\theta_p}\big)\Big)\theta_p}{0.54 + 0.6\big(\frac{\tau_p}{\theta_p}\big)}$

$\gamma = \frac{0.5 \theta_p}{1.27 + 0.6\big(\frac{\tau_p}{\theta_p}\big)}$

### 2.3.3. Tyreus-Luyben Method <a class="anchor" id="section_2_3_3"></a>
The Ziegler-Nichols tuning rules are generally regarded as too aggressive for most process control applications. In 1992, based on collaborative research between the Dow Chemical Company and Lehigh University, Bjorn Tyreus and William Luyben proposed the following modification for the traditional tuning rules. Also, this method only proposes settings for $PI$ and $PID$ controllers.

The rules of this method are summarized in the following table:

| Control Mode | Controller gain $K_c$ | Integral time $\tau_I$ | Derivative time $\tau_D$ |
|:------------:|:---------------------:|:----------------------:|:------------------------:|
| $PI$         | $\frac{K_{cu}}{3.2}$  | $2.2 P_u$              |                          |
| $PID$        | $\frac{K_{cu}}{3.2}$  | $2.2 P_u$              | $\frac{P_u}{6.3}$        |

### 2.3.4. IAE, ITAE Method <a class="anchor" id="section_2_3_4"></a>
In 1967 a research group from Louisiana State University has developed a methodology for minimizing performance criteria based on IAE (Integral Absolute Error) and ITAE (Integral Time Absolute Error). From solving a problem of multi-objective optimization, they obtained a set of rules for adjusting the parameters of the PID controller for different characteristics of a first-order model with dead time.

The rules of this method are summarized in the following tables:

| Control Mode |                      Controller gain $K_c$                     | Integral time $\tau_I$ | Derivative time $\tau_D$ |
|:------------:|:--------------------------------------------------------------:|:----------------------:|:------------------------:|
| $PI$-IAE     | $\frac{0.984}{K_p}\big({\frac{\tau_p}{\theta_p}\big)}^{0.986}$ | $\alpha$               |                          |
| $PI$-ITAE    | $\frac{0.859}{K_p}\big({\frac{\tau_p}{\theta_p}\big)}^{0.977}$ | $\beta$                |                          |
| $PID$-IAE    | $\frac{1.435}{K_p}\big({\frac{\tau_p}{\theta_p}\big)}^{0.921}$ | $\gamma$               | $\varepsilon$               |
| $PID$-ITAE   | $\frac{1.357}{K_p}\big({\frac{\tau_p}{\theta_p}\big)}^{0.947}$ | $\delta$               | $\epsilon$               |

**Where:**

$\alpha = \frac{\tau_p}{0.608\big(\frac{\tau_p}{\theta_p}\big)^{0.707}}$

$\beta = \frac{\tau_p}{0.647\big(\frac{\tau_p}{\theta_p}\big)^{0.680}}$

$\gamma = \frac{\tau_p}{0.878\big(\frac{\tau_p}{\theta_p}\big)^{0.749}}$

$\delta = \frac{\tau_p}{0.842\big(\frac{\tau_p}{\theta_p}\big)^{0.738}}$

$\varepsilon = \tau_p\Big(0.482\big(\frac{\theta_p}{\tau_p}\big)^{1.137}\Big)$

$\epsilon = \tau_p\Big(0.381\big(\frac{\theta_p}{\tau_p}\big)^{0.995}\Big)$

### 2.4. Dynamic process response - FOPDT model <a class="anchor" id="section_2_4"></a>
The methods described above are first-order linear systems. A first-order linear system with time delay is a common empirical description of many stable dynamic processes. The **First Order Plus Dead Time (FOPDT)** model is used to obtain initial controller tuning constants. The mathematical FOPDT equation is:

$$\large\tau_p\frac{d}{dt}y(t) = -y(t)+K_{pr}u(t-\theta_p)\tag{13}$$

**Where:**

$u(t)$ - Input of the controller

$y(t)$ - Output of the controller

$K_{pr}$ - Process gain

$\tau_p$ - Process time constant

$\theta_p$ - Process dead time

<img src="./imgs/fopdt.png" width="350" />

In this equation we have three unknowns. Let's look at them.

### 2.4.1. Process gain <a class="anchor" id="section_2_4_1"></a>
The process gain is the change in the output $y$ induced by a unit change in the input $u$. The process gain is calculated by evaluating the change in $y(t)$ divided by the change in $u(t)$ at steady state initial and final conditions.

$$\large K_{pr} = \frac{\Delta y}{\Delta u}\tag{14}$$

The process gain affects the magnitude of the response, regardless of the speed of response.

### 2.4.2. Process time constant <a class="anchor" id="section_2_4_2"></a>
Given a change in $u(t) = \Delta u$, the solution to the linear first-order differential (without time delay) becomes:

$$\large y(t) = (e^{-\frac{t}{\tau_p}})y(0) + (1 - e^{-\frac{t}{\tau_p}})K_{pr}\Delta u\tag{15}$$

If the initial condition $y(0) = 0$ and at $t = \tau_p$, the solution is simplified to the following:

$$\large y(\tau_p) = (1 - e^{-\frac{\tau_p}{\tau_p}})K_{pr}\Delta u = (1 - e^{-1})K_{pr}\Delta u = 0.632K_{pr}\Delta u\tag{16}$$

The process time constant is therefore the amount of time needed for the output to reach 63.2% of the way to steady state conditions. The process time constant affects the speed of response.

### 2.4.3. Process time delay <a class="anchor" id="section_2_4_3"></a>
The time delay is expressed as a time shift in the input variable $u(t)$.

$$\large u(t - \theta_p)\tag{17}$$

Suppose that that input signal is a step function that normally changes from 0 to 1 at time=0 but this shift is delayed by 5 sec. The input function $u(t)$ and output function $y(t)$ are time-shifted by 5 sec. The solution to the first-order differential equation with time delay is obtained by replacing all variables $t$ with $t - \theta_p$ and applying the conditional result based on the time in relation to the time delay.

$$\large y(t >= \theta_p) = y(0) = (e^{-\frac{t - \theta_p}{\tau_p}})y(0) + (1 - e^{-\frac{t - \theta_p}{\tau_p}})K_{pr}\Delta u\tag{18}$$

For a step change $\Delta u$, the analytical solution for a first-order linear system without time delay $(x(t) = y(t))$ with $\theta_p = 0$

$$\large \tau_p\frac{d}{dt}x(t) = -y(t) + K_{pr} u(t)\tag{19}$$

is

$$\large x(t) = K_{pr}\Big(1 - \exp\Big(-\frac{t}{\tau_p}\Big)\Big)\Delta u\tag{20}$$

With dead-time, the solution becomes:

$$\large y(t) = x(t - \theta_p)S(t - \theta_p) = K_{pr}\Big(1 - \exp\Big(-\frac{t - \theta_p}{\tau_p}\Big)\Big)\Delta u S(t - \theta_p)\tag{21}$$

**Where:**

$S(t - \theta_p)$ - Step function that changes from zero to one at $t = \theta_p$

###  2.4.4. Interactive simulation - FOPDT model <a class="anchor" id="section_2_4_4"></a>
In the following implementation of python is demonstrated how the three parameters $K_{pr}$, $\tau_p$ and $\theta_p$ can affect the FOPDT equation.

In [2]:
# FOPDT model simulation
def fopdt_interactive_simulation(K_pr, tau_p, theta_p):
    time_points = 100  # time points to plot
    t = np.linspace(0, 20, 100)  # create time vector
    # create 0 -> 1 step at t=theta_p
    delay = np.empty_like(t)
    for i in range(time_points):
        if t[i] < theta_p:
            delay[i] = 0.0
        else:
            delay[i] = 1.0
    # calculate response to step input
    x = K_pr * (1.0-np.exp(-(t-theta_p) / tau_p))
    y = x * delay
    # plot response
    plt.figure(1, figsize=(15, 7))
    # plot input and step changes
    plt.subplot(1, 2, 1)
    plt.plot(t, x, c="r", ls="--", lw=2, label=r"$x(t-\theta_p)=K_{pr}(1-\exp(-(t-\theta_p)/\tau_p))$")
    plt.plot(t, delay, c="g", ls=":", lw=2, label=r"$S(t-\theta_p)$")
    plt.title("Input and step changes", fontsize="x-large")
    plt.xlabel("time(sec)", fontsize="large")
    plt.ylabel(r"$u(t)$", fontsize="large")
    plt.legend(loc="best")
    plt.ylim([-10, 10])
    plt.xlim([0, 20])
    plt.grid()
    # plot output response
    plt.subplot(1, 2, 2)
    plt.plot(t, y, c="k", ls='-', linewidth=4, label=r"$y(t)=x(t-\theta)S(t-\theta)$")
    plt.title("Output response", fontsize="x-large")
    plt.xlabel("time(sec)", fontsize="large")
    plt.ylabel(r"$y(t)$", fontsize="large")
    plt.legend(loc="best")
    plt.ylim([-10, 10])
    plt.xlim([0, 20])
    plt.grid()


K_pr_slide = wg.FloatSlider(value=3.0, min=-10.0, max=10.0, step=0.1)
tau_p_slide = wg.FloatSlider(value=4.0, min=0.1, max=10.0, step=0.1)
theta_p_slide = wg.FloatSlider(value=5.0, min=0.1, max=10.0, step=0.1)
wg.interact(fopdt_interactive_simulation, K_pr=K_pr_slide, tau_p=tau_p_slide, theta_p=theta_p_slide)
print("FOPDT Simulator: Adjust K_pr, tau_p and theta_p")

interactive(children=(FloatSlider(value=3.0, description='K_pr', max=10.0, min=-10.0), FloatSlider(value=4.0, …

FOPDT Simulator: Adjust K_pr, tau_p and theta_p


### 3. Interactive simulations of PID controller tuning <a class="anchor" id="chapter3"></a>
In the following cells are implemented interactive simulations respectively of $P$, $PI$, $PID$ controllers. The system that will be controlled is a temperature process. The initial temperature of the object is assumed to be $20^oC$. The values of the parameters of the FOPDT model are chosen to be as close as possible to a real-life process ($K_{pr} = 1 [\frac{^oC}{\%}],\tau_p = 120.0 [sec],\theta_p = 15.0 [sec]$). The time for which will simulate the system is 10 minutes (600 seconds).

In [3]:
# Simulation time
time_points = 601  # time points to plot
time_final = 600.0  # final time

# FOPDT parameters
Kp = 1
tau_p = 120.0
theta_p = 15.0

# Temperatures and times for simulation
temp_init = 20.0
desired_temp_1 = 60.0
desired_temp_2 = 40.0
desired_time_1 = 10
desired_time_2 = 300


# Process simulation
def process_simulation(y, t, u):
    dydt = (-(y-temp_init)+Kp * u) / tau_p
    return dydt

### 3.1. P control tuning <a class="anchor" id="section_3_1"></a>
In the following implementation is shown an interactive simulation of the $P$ controller tuning. The aim is to reach the desired temperature $(60^oC)$ in 10 seconds, using a heater and quantify the offset between the desired target and the measured temperature.

In [4]:
# P control
def p_control_interactive_simulation(Kc):
    t = np.linspace(0, time_final, time_points)  # create time vector
    P = np.zeros(time_points)  # initialize proportional term
    e = np.zeros(time_points)  # initialize error
    OP = np.zeros(time_points)  # initialize controller output
    PV = np.ones(time_points) * temp_init  # initialize process variable
    SP = np.ones(time_points) * temp_init  # initialize set point
    SP[desired_time_1:] = desired_temp_1  # step up
    y0 = temp_init  # initial condition
    iae = 0.0  # initialize integral abs error
    # loop through all time steps
    for i in range(1, time_points):
        # simulate process for one time step
        time_interval = [t[i-1], t[i]]  # time interval
        y = odeint(process_simulation, y0, time_interval, args=(OP[max(0, i-1-int(theta_p))],))  # compute next step
        y0 = y[1]  # record new initial condition
        iae += np.abs(SP[i]-y0[0])  # calculate integral abs error
        # calculate new OP with PID
        PV[i] = y[1]  # record PV
        e[i] = SP[i]-PV[i]  # calculate error
        P[i] = Kc * e[i]  # calculate proportional term
        OP[i] = P[i]  # calculate new controller output
        # controller output limitation
        if OP[i] >= 100:
            OP[i] = 100.0
        if OP[i] <= 0:
            OP[i] = 0.0
    # plot PID response
    plt.figure(1, figsize=(15, 7))
    # plot setpoint and measured temperature
    plt.subplot(2, 2, 1)
    plt.plot(t, SP, c="k", ls='-', lw=2, label="Setpoint (SP)")
    plt.plot(t, PV, c="r", ls=':', lw=2, label="Temperature (PV)")
    plt.xlabel("time (sec)")
    plt.ylabel(r"T $(^oC)$")
    plt.text(200, 40, f"Offset: {np.round(SP[-1]-PV[-1], 2)}")
    plt.text(200, 35, f"$K_c$: {np.round(Kc, 0)}")
    plt.legend(loc="best")
    # plot proportional term
    plt.subplot(2, 2, 2)
    plt.plot(t, P, c="g", ls="-", lw=2, label=r"Proportional = $K_c e(t)$")
    plt.xlabel("time (sec)")
    plt.ylabel("Amplitude")
    plt.legend(loc="best")
    # plot error
    plt.subplot(2, 2, 3)
    plt.plot(t, e, c="m", ls="--", lw=2, label="Error (e=SP-PV)")
    plt.xlabel("time (sec)")
    plt.ylabel(r"$\Delta T$ $(^oC)$")
    plt.legend(loc="best")
    # plot output (heater)
    plt.subplot(2, 2, 4)
    plt.plot(t, OP, c="b", ls="--", lw=2, label="Heater (OP)")
    plt.xlabel("time (sec)")
    plt.ylabel("Output (%)")
    plt.legend(loc="best")
    
    
Kc_slide = wg.FloatSlider(value=10.0, min=0.0, max=20.0, step=1.0)
wg.interact(p_control_interactive_simulation, Kc=Kc_slide)
print('P Simulator: Adjust Kc and Calculate Offset')

interactive(children=(FloatSlider(value=10.0, description='Kc', max=20.0, step=1.0), Output()), _dom_classes=(…

P Simulator: Adjust Kc and Calculate Offset


### 3.2. PI control tuning <a class="anchor" id="section_3_2"></a>
In the following implementation is shown an interactive simulation of the $PI$ controller tuning. The aim is to reach the desired temperature $(60^oC)$ in 10 seconds, using a heater and calculated integral absolute error.

In [5]:
# PI control
def pi_control_interactive_simulation(Kc, tau_I):
    t = np.linspace(0, time_final, time_points)  # create time vector
    P = np.zeros(time_points)  # initialize proportional term
    I = np.zeros(time_points)  # initialize integral term
    e = np.zeros(time_points)  # initialize error
    OP = np.zeros(time_points)  # initialize controller output
    PV = np.ones(time_points) * temp_init  # initialize process variable
    SP = np.ones(time_points) * temp_init  # initialize setpoint
    SP[desired_time_1:] = desired_temp_1  # step up
    y0 = temp_init  # initial condition
    iae = 0.0  # initialize integral abs error
    # loop through all time steps
    for i in range(1, time_points):
        # simulate process for one time step
        ts = [t[i-1], t[i]]  # time interval
        y = odeint(process_simulation, y0, ts, args=(OP[max(0, i-int(theta_p))],))  # compute next step
        y0 = y[1]  # record new initial condition
        iae += np.abs(SP[i]-y0[0])  # calculate integral abs error
        # calculate new OP with PID
        PV[i] = y[1]  # record PV
        e[i] = SP[i]-PV[i]  # calculate error = SP - PV
        dt = t[i]-t[i-1]  # calculate time step
        P[i] = Kc * e[i]  # calculate proportional term
        I[i] = I[i-1]+(Kc / tau_I) * e[i] * dt  # calculate integral term
        OP[i] = P[i]+I[i]  # calculate new controller output
        # controller output limitation
        if OP[i] >= 100:
            OP[i] = 100.0
            I[i] = I[i-1]  # reset integral
        if OP[i] <= 0:
            OP[i] = 0.0
            I[i] = I[i-1]  # reset integral
    # plot PID response
    plt.figure(1, figsize=(15, 7))
    # plot setpoint and measured temperature
    plt.subplot(2, 2, 1)
    plt.plot(t, SP, c="k", ls="-", lw=2, label="Setpoint (SP)")
    plt.plot(t, PV, c="r", ls=":", lw=2, label="Temperature (PV)")
    plt.xlabel("time (sec)")
    plt.ylabel(r"T $(^oC)$")
    plt.text(200, 40, f"Integral Abs Error: {np.round(iae, 2)}")
    plt.text(200, 35, f"$K_c$: {np.round(Kc, 0)}")
    plt.text(200, 30, f"$\\tau_I$: {np.round(tau_I, 0)} sec")
    plt.legend(loc="best")
    # plot terms
    plt.subplot(2, 2, 2)
    plt.plot(t, P, 'g.-', linewidth=2, label=r"Proportional = $K_c e(t)$")
    plt.plot(t, I, 'b-', linewidth=2, label=r"Integral = $\frac{K_c}{\tau_I} \int_{i=0}^{n_t} e(t)dt$")
    plt.xlabel("time (sec)")
    plt.ylabel("Amplitude")
    plt.legend(loc="best")
    # plot error
    plt.subplot(2, 2, 3)
    plt.plot(t, e, c="m", ls="--", lw=2, label="Error (e=SP-PV)")
    plt.xlabel('time (sec)')
    plt.ylabel(r"$\Delta T$ $(^oC)$")
    plt.legend(loc="best")
    # plot output
    plt.subplot(2, 2, 4)
    plt.plot(t, OP, c="b", ls="--", lw=2, label="Heater (OP)")
    plt.xlabel("time (sec)")
    plt.ylabel("Output (%)")
    plt.legend(loc="best")


Kc_slide = wg.FloatSlider(value=5.0, min=0.0, max=20.0, step=1.0)
tau_I_slide = wg.FloatSlider(value=50.0, min=1.0, max=180.0, step=1.0)
wg.interact(pi_control_interactive_simulation, Kc=Kc_slide, tau_I=tau_I_slide)
print('PI Simulator: Adjust Kc and tau_I for lowest Integral Abs Error')

interactive(children=(FloatSlider(value=5.0, description='Kc', max=20.0, step=1.0), FloatSlider(value=50.0, de…

PI Simulator: Adjust Kc and tau_I for lowest Integral Abs Error


### 3.3. PID control tuning <a class="anchor" id="section_3_3"></a>
In the following implementation is shown an interactive simulation of the $PID$ controller tuning. The aim is to reach the desired temperature $60^oC$ in 10 seconds and to maintain it for up to 300 seconds. Then the temperature should drop to $40^oC$ and be maintained until the end of the process.

In [6]:
# PID control
def pid_control_interactive_simulation(Kc, tau_I, tau_D):
    t = np.linspace(0, time_final, time_points)  # create time vector
    P = np.zeros(time_points)  # initialize proportional term
    I = np.zeros(time_points)  # initialize integral term
    D = np.zeros(time_points)  # initialize derivative term
    e = np.zeros(time_points)  # initialize error
    OP = np.zeros(time_points)  # initialize controller output
    PV = np.ones(time_points) * temp_init  # initialize process variable
    SP = np.ones(time_points) * temp_init  # initialize setpoint
    SP[desired_time_1:desired_time_2] = desired_temp_1  # step up
    SP[desired_time_2:] = desired_temp_2  # step down
    y0 = temp_init  # initial condition
    iae = 0.0  # initialize integral abs error
    # loop through all time steps
    for i in range(1, time_points):
        # simulate process for one time step
        ts = [t[i-1], t[i]]  # time interval
        y = odeint(process_simulation, y0, ts, args=(OP[max(0, i-int(theta_p))],))  # compute next step
        y0 = y[1]  # record new initial condition
        iae += np.abs(SP[i]-y0[0])  # calculate integral abs error
        # calculate new OP with PID
        PV[i] = y[1]  # record PV
        e[i] = SP[i]-PV[i]  # calculate error = SP - PV
        dt = t[i]-t[i-1]  # calculate time step
        P[i] = Kc * e[i]  # calculate proportional term
        I[i] = I[i-1]+(Kc / tau_I) * e[i] * dt  # calculate integral term
        D[i] = -Kc * tau_D * (PV[i]-PV[i-1]) / dt  # calculate derivative term
        OP[i] = P[i]+I[i]+D[i]  # calculate new controller output
        # controller output limitation
        if OP[i] >= 100:
            OP[i] = 100.0
            I[i] = I[i-1]  # reset integral
        if OP[i] <= 0:
            OP[i] = 0.0
            I[i] = I[i-1]  # reset integral
    # plot PID response
    plt.figure(1, figsize=(15, 7))
    # plot setpoint and measured temperature
    plt.subplot(2, 2, 1)
    plt.plot(t, SP, c="k", ls="-", lw=2, label="Setpoint (SP)")
    plt.plot(t, PV, c="r", ls=":", lw=2, label="Temperature (PV)")
    plt.xlabel("time (sec)")
    plt.ylabel(r"T $(^oC)$")
    plt.text(100, 32, f"Integral Abs Error: {np.round(iae, 2)}")
    plt.text(400, 32, f"$K_c$: {np.round(Kc, 0)}")
    plt.text(400, 27, f"$\\tau_I$: {np.round(tau_I, 0)} sec")
    plt.text(400, 22, f"$\\tau_D$: {np.round(tau_D, 0)} sec")
    plt.legend(loc="best")
    # plot terms
    plt.subplot(2, 2, 2)
    plt.plot(t, P, c="g", ls="-.", lw=2, label=r"Proportional = $K_c e(t)$")
    plt.plot(t, I, c="b", ls="-", lw=2, label=r"Integral = $\frac{K_c}{\tau_I} \int_{i=0}^{n_t} e(t)dt$")
    plt.plot(t, D, c="r", ls="--", lw=2, label=r"Derivative = $-K_c \tau_D \frac{d(PV)}{dt}$")
    plt.xlabel("time (sec)")
    plt.ylabel("Amplitude")
    plt.legend(loc="best")
    # plot error
    plt.subplot(2, 2, 3)
    plt.plot(t, e, c="m", ls="--", lw=2, label="Error (e=SP-PV)")
    plt.xlabel("time (sec)")
    plt.ylabel(r"$\Delta T$ $(^oC)$")
    plt.legend(loc="best")
    # plot output
    plt.subplot(2, 2, 4)
    plt.plot(t, OP, c="b", ls="--", lw=2, label="Heater (OP)")
    plt.xlabel("time (sec)")
    plt.ylabel("Output (%)")
    plt.legend(loc="best")


Kc_slide = wg.FloatSlider(value=6.0, min=0.0, max=20, step=1.0)
tau_I_slide = wg.FloatSlider(value=45.0, min=0.1, max=180.0, step=1.0)
tau_D_slide = wg.FloatSlider(value=3.0, min=0.0, max=20.0, step=1.0)
wg.interact(pid_control_interactive_simulation, Kc=Kc_slide, tau_I=tau_I_slide, tau_D=tau_D_slide)
print('PID Simulator: Adjust Kc, tau_I, and tau_D to achieve lowest Integral Abs Error')

interactive(children=(FloatSlider(value=6.0, description='Kc', max=20.0, step=1.0), FloatSlider(value=45.0, de…

PID Simulator: Adjust Kc, tau_I, and tau_D to achieve lowest Integral Abs Error


### 4. Conclusion <a class="anchor" id="chapter4"></a>
The objective of the simulation is to demonstrate the characteristics of $P$, $PI$, and $PID$ controllers on closed loop temperature control. 

Upon using the $P$ controller, it shows a relatively quick response, but with an offset. At a higher $K_c$ value, the $P$ controller reacts faster but if it is too high, oscillatory behavior will start to show. Also, the higher the temperature deviation for the P controller, the less the error percentage, but it will take more time. As demonstrated in the simulation, this is the least favorable controller for temperature sensitive systems as it will take too long to correct the error and may lead to serious problems during the process.

Then with the $PI$ controller the reaction is ultimately oscillating (pseudostationary state) due to the slow response of the integral action. If the process has more time, the PI controller will eliminate the compensation and reach a new stable temperature.

The conclusion is that the best type of controller is the $PID$ controller. Its response is faster than the $PI$ controller with less oscillation, thanks to the derivative controller, which fixes the response time to compensate for the slowness of its integral action. The integrated action also eliminates compensation, thus making the $PID$ controller a faster, less flickering and accurate controller for correcting any temperature-sensitive system.

### 5. References <a class="anchor" id="chapter5"></a>
1. Jens Graf - "PID Control Fundamentals" Aug 7, 2016
2. Tore Hagglund (Karl J. Astrom and Tore Hagglund) - "PID Controllers: Theory, Design, and Tuning - second edition" Jan 1, 1995
3. David W. St. Clair - "Controller Tuning and Control Loop Performance" Jan 1, 1990
4. <a href="https://en.wikipedia.org/wiki/PID_controller">Wikipedia - PID controller</a>
5. <a href="https://apmonitor.com/pdc/index.php/Main/HomePage">Brigham Young University (Provo, Utah) - Process Dynamics and Control in Python</a>
6. <a href="https://jckantor.github.io/CBE30338/">University of Notre Dame - Chemical Process Control</a>
7. <a href="https://studentnet.cs.manchester.ac.uk/resources/library/thesis_abstracts/MSc14/FullText/Ioannidis-Feidias-fulltext.pdf">University of Manchester - Intelligent controller based on Raspberry PI </a>
8. <a href="https://www.youtube.com/watch?v=3viD5ij60EI">ABB Process Automation - Tuning A Control Loop </a>
9. <a href="https://www.intechopen.com/books/pid-control-for-industrial-processes/advanced-methods-of-pid-controller-tuning-for-specified-performance">Intech Open - Advanced methods of PID controller tuning</a>